In [1]:
import polars as pl
import hawk_backtester as hbt

ModuleNotFoundError: No module named 'polars'

Load Model State


In [ ]:
model_state = pl.read_csv("data/example_ms.csv")
# Coerce date column to datetime
model_state = model_state.with_columns(
    pl.col("date").str.strptime(pl.Datetime, "%Y-%m-%d"),
    pl.col("rolling_hurst_exp_126").cast(pl.Float64)
)
model_state.tail()

date,ticker,open,high,low,close,volume,open_interest,ema_close_8,ema_close_16,ema_close_32,ema_close_64,ema_close_128,ema_close_256,rsi_close_14,return_close_1,return_close_5,return_close_8,return_close_10,return_close_16,return_close_20,return_close_32,return_close_50,return_close_64,return_close_128,return_close_256,vol_ratio,mean_rev,vpt,adaptive_mom,rolling_hurst_exp_126,rolling_hurst_exp_252,rolling_hurst_exp_504,adaptive_kernel_trend,kernel_momentum_div,kernel_slope,ema_spread_16_64,cmse_100,spectral_entropy,wavelet_mom,close_volatility_20
datetime[μs],str,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str,str,f64,i64,f64,f64,f64,f64,f64,f64
2025-02-11 00:00:00,"""Z00-IFEU""",10837.459438,10857.281037,10804.629914,10846.750812,60710,478009,10773.599461,10676.485287,10570.676847,10537.274254,"""10530.54592151126""","""10383.010931813""",62.766854,-0.001863,0.021861,0.010046,0.019659,0.043213,0.088347,0.065588,0.031061,0.021061,"""0.03911521834899201""","""0.13963225003797586""",0.527216,-0.022627,-303874.8746,0.016928,0.727762,"""0.6915116864359778""","""0.6710873582316578""",0.0,-1,null,139.211033,6.932235,0.098488,44.507938,0.008839
2025-02-12 00:00:00,"""Z00-IFEU""",10858.972032,10921.002033,10830.438231,10893.088533,67209,484336,10800.152588,10701.968022,10590.216949,10548.222386,"""10536.16673719064""","""10386.980407307306""",64.421428,0.004272,0.01561,0.008716,0.025448,0.036478,0.075322,0.067003,0.034828,0.041271,"""0.037094258180448536""","""0.15611600731204156""",0.921355,-0.023315,-303738.834719,0.023532,0.73648,"""0.679904871723681""","""0.6708275376742439""",0.0,-1,null,153.745636,6.933857,0.09849,45.70187,0.008364
2025-02-13 00:00:00,"""Z00-IFEU""",10954.716448,10962.176248,10822.926646,10859.603997,89656,486375,10813.364012,10720.513431,10606.543437,10557.803358,"""10541.181268346429""","""10390.658411894323""",62.26807,-0.003074,-0.004242,0.021598,0.011243,0.033732,0.056384,0.061857,0.028321,0.036161,"""0.027608043765138257""","""0.14980275393143927""",1.3987,-0.017399,-303646.478729,0.013364,0.727868,"""0.6813213757136763""","""0.6774485170970898""",0.0,-1,null,162.710072,6.933175,0.098489,44.492102,0.008062
2025-02-14 00:00:00,"""Z00-IFEU""",11019.105362,11032.354788,10946.548986,10971.155061,68232,492147,10848.42869,10750.000681,10628.641111,10570.521872,"""10547.847528702272""","""10395.175895547272""",66.308439,0.010272,0.0189,0.033581,0.015945,0.040169,0.0542,0.071629,0.04362,0.069054,"""0.030493002923891233""","""0.1507069117679667""",0.890538,-0.025141,-303506.159031,0.014711,0.734938,"""0.6839772570978544""","""0.6708039079040017""",0.0,-1,null,179.478809,6.932795,0.09849,47.501503,0.007944
2025-02-17 00:00:00,"""Z00-IFEU""",10958.536561,11048.758838,10947.810836,11029.200163,42235,489365,10888.600128,10782.847679,10652.917417,10584.63505,"""10555.310360234771""","""10400.109936535608""",68.215663,0.005291,0.014927,0.028301,0.037553,0.040595,0.06076,0.077679,0.038815,0.077242,"""0.0250078694438447""","""0.13991157389989883""",1.037026,-0.027532,-303362.920902,0.023391,0.735566,"""0.6866209728308709""","""0.6678336507595046""",0.0,-1,null,198.212629,6.903697,0.09849,43.178145,0.007917


Model State Contains the following columns:

- date: the date corresponding to the record
- ticker: the ticker symbol of the security (futures contract in our case)

Base Dataset Feature Columns:

- open: the opening price of the security
- high: the highest price of the security
- low: the lowest price of the security
- close: the closing price of the security
- volume: the volume of the security
- open_interest: the open interest of the security

Basic Computed Features:

- EMA's
- RSI
- Lag Returns

Custom Computed Features:

- Kernel Regression Features
- Hurst Exponent
- Wavlet Momentum
- Spectral Entropy
- Etc.

The idea is to add unique features to the dataset that can help build better models.


### Example Model

- For the demonstration we will build and backtest a simple model.
- We will use a subset of the features provided in the example model state, and add our own custom features.


### Adding Model Features

Vix Feature Example:
https://my.apps.factset.com/workstation/navigator/company-security/price-history/VX00-USA


In [ ]:
vix_data = pl.read_csv("data/vix_data.csv")
# Fix Column types
vix_data = vix_data.with_columns(
    pl.col("Date").str.strptime(pl.Datetime, "%m/%d/%y"),
    pl.col("Last").cast(pl.Float64),
    pl.col("Settlement Price").cast(pl.Float64),
    pl.col("Change").cast(pl.Float64),
    pl.col("% Change").cast(pl.Float64),
    pl.col("Bid").cast(pl.Float64),
    pl.col("Ask").cast(pl.Float64),
    pl.col("Open Interest").str.replace(",", "").cast(pl.Float64),
    pl.col("CVol").cast(pl.Int64)
)


vix_data.head()

Date,Last,Settlement Price,Change,% Change,Bid,Ask,Open Interest,CVol
datetime[μs],f64,f64,f64,f64,f64,f64,f64,i64
2025-02-21 00:00:00,18.2,18.3,1.17,6.85,0.0,0.0,211031.0,162044
2025-02-20 00:00:00,16.95,17.03,0.13,0.75,16.95,17.0,215031.0,90335
2025-02-19 00:00:00,17.0,16.91,-0.22,-1.28,16.95,17.0,189988.0,73695
2025-02-18 00:00:00,17.13,17.13,-0.06,-0.33,17.1,17.15,174006.0,107276
2025-02-14 00:00:00,17.21,17.18,1.05,6.49,17.2,17.25,161220.0,73101


Select important columns. We are interested in close.


In [ ]:
vix_data = vix_data.select(["Settlement Price", "Date"])
vix_data.head()

# Rename columns to match model state
vix_data = vix_data.rename({"Date": "date", "Settlement Price": "vix_close"})
vix_data.head()


vix_close,date
f64,datetime[μs]
18.3,2025-02-21 00:00:00
17.03,2025-02-20 00:00:00
16.91,2025-02-19 00:00:00
17.13,2025-02-18 00:00:00
17.18,2025-02-14 00:00:00


Now lets merge the vix data with the model state.


In [ ]:
# We are interested in NQ00-USA, select this ticker from the model state
nq_data = model_state.filter(pl.col("ticker") == "NQ00-USA")
nq_data.head()

date,ticker,open,high,low,close,volume,open_interest,ema_close_8,ema_close_16,ema_close_32,ema_close_64,ema_close_128,ema_close_256,rsi_close_14,return_close_1,return_close_5,return_close_8,return_close_10,return_close_16,return_close_20,return_close_32,return_close_50,return_close_64,return_close_128,return_close_256,vol_ratio,mean_rev,vpt,adaptive_mom,rolling_hurst_exp_126,rolling_hurst_exp_252,rolling_hurst_exp_504,adaptive_kernel_trend,kernel_momentum_div,kernel_slope,ema_spread_16_64,cmse_100,spectral_entropy,wavelet_mom,close_volatility_20
datetime[μs],str,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str,str,f64,i64,f64,f64,f64,f64,f64,f64
2010-01-04 00:00:00,"""NQ00-USA""",1865.5,1889.0,1865.5,1886.75,205157,303786,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0,null,null,null,null,null,null
2010-01-05 00:00:00,"""NQ00-USA""",1886.5,1890.25,1876.0,1885.25,208698,314877,null,null,null,null,null,null,null,-0.000795,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0,null,null,null,null,null,null
2010-01-06 00:00:00,"""NQ00-USA""",1885.25,1891.75,1872.25,1878.5,261274,309080,null,null,null,null,null,null,null,-0.00358,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0,null,null,null,null,null,null
2010-01-07 00:00:00,"""NQ00-USA""",1877.75,1880.5,1865.75,1877.5,241321,310532,null,null,null,null,null,null,null,-0.000532,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0,null,null,null,null,null,null
2010-01-08 00:00:00,"""NQ00-USA""",1877.5,1892.5,1865.0,1890.0,268934,317226,null,null,null,null,null,null,null,0.006658,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0,null,null,null,null,null,null


In [ ]:
# merge the vix data with the nq data
nq_data = nq_data.join(vix_data, on="date", how="left")
nq_data.head()
# Fill NaN values with 0
nq_data = nq_data.fill_nan(0)
nq_data.tail()


date,ticker,open,high,low,close,volume,open_interest,ema_close_8,ema_close_16,ema_close_32,ema_close_64,ema_close_128,ema_close_256,rsi_close_14,return_close_1,return_close_5,return_close_8,return_close_10,return_close_16,return_close_20,return_close_32,return_close_50,return_close_64,return_close_128,return_close_256,vol_ratio,mean_rev,vpt,adaptive_mom,rolling_hurst_exp_126,rolling_hurst_exp_252,rolling_hurst_exp_504,adaptive_kernel_trend,kernel_momentum_div,kernel_slope,ema_spread_16_64,cmse_100,spectral_entropy,wavelet_mom,close_volatility_20,vix_close
datetime[μs],str,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str,str,f64,i64,f64,f64,f64,f64,f64,f64,f64
2025-02-11 00:00:00,"""NQ00-USA""",21832.0,21871.5,21669.5,21786.75,418850,261596,21732.033573,21671.516833,21586.275405,21338.722188,"""20763.29340189567""","""19635.774027208365""",54.325473,-0.002735,0.017835,0.012243,0.024849,0.008891,0.040077,0.010213,0.037786,0.042728,"""0.21263184259594237""","""0.2286165428374054""",0.560916,-0.007051,-448195.013267,0.024124,0.649794,"""0.6567427574970319""","""0.654996088134111""",0.0,-1,null,332.794645,2.678827,0.181019,128.460089,0.010771,16.77
2025-02-12 00:00:00,"""NQ00-USA""",21794.25,21862.5,21510.75,21804.75,541370,260552,21748.192779,21687.191323,21599.516289,21353.061506,"""20779.440015819768""","""19652.653217658106""",54.735369,0.000826,0.006207,0.008324,0.010333,0.009725,0.042317,0.002367,0.047651,0.027327,"""0.1770286500856939""","""0.23190677966101703""",0.99713,-0.005825,-448695.538952,0.012374,0.637723,"""0.661023262998925""","""0.6566902921773436""",0.0,-1,null,334.129818,2.691876,0.181057,114.678578,0.010746,16.8
2025-02-13 00:00:00,"""NQ00-USA""",21861.25,22124.0,21753.0,22113.25,529604,265149,21829.316606,21737.315874,21630.651666,21376.451921,"""20800.1192403807""","""19671.80183075026""",61.16759,0.014148,0.016094,0.024271,0.027412,0.018598,0.033317,0.003847,0.053338,0.041543,"""0.18786259131929528""","""0.2521659116647792""",1.03393,-0.018381,-448724.507321,0.031307,0.628007,"""0.6476557875210672""","""0.6538555843208582""",0.0,0,null,360.863953,2.690843,0.181097,132.080476,0.01,16.13
2025-02-14 00:00:00,"""NQ00-USA""",22071.0,22225.0,22042.5,22196.25,429521,276307,21910.85736,21791.308124,21664.930353,21401.676477,"""20821.764678514333""","""19691.447341795007""",62.703201,0.003753,0.014836,0.036966,0.026428,0.008898,0.027854,0.022939,0.046105,0.046117,"""0.19081788674588918""","""0.24406238177308848""",0.53008,-0.020789,-448316.250179,0.02568,0.619835,"""0.6471110967781114""","""0.645940741664095""",0.0,0,null,389.631647,2.688947,0.18115,110.676353,0.009862,17.18
2025-02-17 00:00:00,"""NQ00-USA""",22235.75,22281.75,22192.5,22196.25,101224,276307,21974.277947,21838.948344,21697.131543,21426.124893,"""20843.07452845985""","""19710.939969485316""",62.703201,0.0,0.016009,0.024273,0.028116,0.007169,0.027854,0.036421,0.042983,0.047512,"""0.16184880327675777""","""0.24221734081401358""",0.265516,-0.01938,-446903.967856,0.007134,0.626941,"""0.6478880827252453""","""0.6546064022100112""",0.0,1,null,412.823451,2.667274,0.181207,130.284424,0.009862,null


For the sake ofsimplicity, we will just select the featurew we are interested in.


In [ ]:
# Select important columns
nq_data = nq_data.select([
    "date",
    "ticker",
    "close",
    "volume",
    "open_interest",
    "ema_close_8",
    "ema_close_16",
    "ema_close_32",
    "ema_close_64",
    "ema_close_128",
    "ema_close_256",
    "rsi_close_14",
    "return_close_1",
    "return_close_5",
    "return_close_8",
    "return_close_16",
    "return_close_32",
    "return_close_64",
    "return_close_128",
    "return_close_256",
    "vix_close",
    "close_volatility_20",
    "rolling_hurst_exp_126"
])


In [ ]:
### Backtesting Price Data
prices_df = hbt.prepare_price_data(nq_data)
prices_df
#prices_df.head()

NameError: name 'hbt' is not defined

### Trading Model

Now we will build a simple trading model. We will use the features we have built to make trading decisions.
We will generate 'portfolio weights' based on our decisions process.


Simplest example:

- If short ema_close_8 is greater than long ema_close_16, then we will go long.
- If long ema_close_8 is greater than short ema_close_16, then we will go short.
- Otherwise, we will hold cash.


In [ ]:
def ema_cross_model(model_state: pl.DataFrame) -> pl.DataFrame:
    """
    Construct insights from the model state using most recent EMAs.
    Returns a DataFrame with date and trading weight (+1 for long, -1 for short, 0 for flat)
    """
    # Get the most recent date's data
    latest_date = model_state["date"].max()
    latest_data = model_state.filter(pl.col("date") == latest_date)

    # Check if we have data for the latest date
    if len(latest_data) == 0:
        return pl.DataFrame({
            "date": [latest_date],
            "weight": [0.0]
        })

    # Check for any null values in the required columns before extracting values
    null_check = latest_data.select([
        pl.col("ema_close_8").is_null(),
        pl.col("ema_close_16").is_null(),
        pl.col("date").is_null(),
        pl.col("vix_close").is_null(),
        pl.col("rolling_hurst_exp_126").is_null()
    ]).to_series().any()

    if null_check:
        return pl.DataFrame({
            "date": [latest_date],
            "weight": [0.0]
        })

    # Now safely get the values
    try:
        short_ema = float(latest_data.get_column("ema_close_8")[0])
        long_ema = float(latest_data.get_column("ema_close_16")[0])
        vix_close = float(latest_data.get_column("vix_close")[0])
        rolling_hurst_exp_126 = float(latest_data.get_column("rolling_hurst_exp_126")[0])
    except (IndexError, ValueError, TypeError):
        return pl.DataFrame({
            "date": [latest_date],
            "weight": [0.0]
        })

    # Generate trading signal
    if short_ema > long_ema and rolling_hurst_exp_126 > 0.5 and vix_close < 16:
        weight = 1.0  # Long signal
    elif short_ema < long_ema and rolling_hurst_exp_126 > 0.5 and vix_close > 16:
        weight = -1.0  # Short signal
    else:
        weight = 0.0  # No position

    # Return insights for the latest date
    return pl.DataFrame({
        "date": [latest_date],
        "weight": [weight]
    })

In [ ]:
BACKTEST_START_DATE = pl.datetime(2015, 1, 1)

def backfill_portfolio_weights(start_date, model_func, price_data):
    dates = (price_data
        .filter(pl.col("date") >= start_date)
        .get_column("date")
        .unique()
        .sort()
    )

    all_insights = []

    for date in dates:
        historical_slice = price_data.filter(pl.col("date") <= date)
        insights = model_func(historical_slice)
        all_insights.append(insights)


    weights = pl.concat(all_insights)
    weights = weights.rename({"weight": "NQ00-USA"})
    weights = weights.rename({"date": "insight_date"})
    weights = hbt.prepare_weight_data(weights)
    return weights

def run_backtest(prices, weights):
    # Align dates of prices and weights
    # Extract unique dates from prices and weights
    prices_dates = set(prices.get_column("date").unique())
    weights_dates = set(weights.get_column("insight_date").unique())

    # Find intersection of dates using set intersection
    shared_dates = list(prices_dates.intersection(weights_dates))

    # Filter prices and weights to only include shared dates
    prices = prices.filter(pl.col("date").is_in(shared_dates))
    weights = weights.filter(pl.col("insight_date").is_in(shared_dates))

    backtester = hbt.PortfolioBacktester(
    prices=prices, weights=weights, risk_free_rate=0.0
    )

    performance = backtester.run()
    metrics = backtester.calculate_metrics()
    return performance, metrics

def plot_results(results: pl.DataFrame) -> alt.Chart:
    # Change date type from i64 epoch time to datetime
    results = results.with_columns(pl.col("date").cast(pl.Date))

    chart = alt.Chart(results).mark_line().encode(
        x='date:T',  # :T specifies temporal (time) data
        y='cumulative_returns:Q',  # :Q specifies quantitative data
        tooltip=['date:T', 'cumulative_returns:Q']
    ).properties(
        width=800,
        height=400,
        title='Cumulative Returns'
    )
    return chart

In [ ]:
# Backtest Benchmark Model
def benchmark_model(model_state: pl.DataFrame) -> pl.DataFrame:
    """
    Construct insights from the model state using most recent EMAs.
    Returns a DataFrame with date and trading weight (+1 for long, -1 for short, 0 for flat)
    """
    return pl.DataFrame({"date": [model_state["date"].max()], "weight": [1.0]})

benchmark_weights = backfill_portfolio_weights(BACKTEST_START_DATE, benchmark_model, nq_data)
benchmark_weights

benchmark_performance, benchmark_metrics = run_backtest(prices_df, benchmark_weights)
print(benchmark_metrics)
plot_results(benchmark_performance)

Insights Date type: Datetime(time_unit='us', time_zone=None)
{'annual_return': 0.17912433146182694, 'annual_volatility': 0.2182971773274107, 'sharpe_ratio': 0.8205526688655677, 'max_drawdown': -0.352760180995476, 'total_return': 4.266951414842495}



thread 'polars-0' panicked at /Users/runner/work/polars/polars/crates/polars-arrow/src/temporal_conversions.rs:170:37:
invalid or out-of-range datetime
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: invalid or out-of-range datetime

In [ ]:
# Backfill the portfolio weights and run the backtest for the ema cross model
ema_cross_weights = backfill_portfolio_weights(BACKTEST_START_DATE, ema_cross_model, nq_data)
ema_cross_weights

ema_cross_performance, ema_cross_metrics = run_backtest(prices_df, ema_cross_weights)
print(ema_cross_metrics)
plot_results(ema_cross_performance)

Insights Date type: Datetime(time_unit='us', time_zone=None)
{'annual_return': -0.004196669137159215, 'annual_volatility': 0.18224512375274432, 'sharpe_ratio': -0.023027607272790032, 'max_drawdown': -0.3287032235602113, 'total_return': -0.041518921089882865}


alt.Chart(...)

In [ ]:
ema_cross_performance

date,returns,cumulative_returns
i64,f64,f64
1420416000,0.0,0.0
1420502400,0.0144,0.0144
1420588800,-0.011934,0.002466
1420675200,-0.019264,-0.016798
1421020800,0.015657,-0.001141
…,…,…
1739232000,0.0,-0.042405
1739318400,0.0,-0.042405
1739404800,0.0,-0.042405


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

def random_forest_model(model_state: pl.DataFrame) -> pl.DataFrame:
    """
    Construct insights using a random forest model.
    Returns a DataFrame with date and trading weight (+1 for long, -1 for short, 0 for flat)
    """
    # Get the most recent date's data
    latest_date = model_state["date"].max()
    latest_data = model_state.filter(pl.col("date") == latest_date)

    # Check if we have enough historical data (need at least 252 days)
    if len(model_state) < 252:
        return pl.DataFrame({
            "date": [latest_date],
            "weight": [0.0]
        })

    # Create target variable (1 if 5-day return > 0, -1 if < 0)
    historical = model_state.with_columns(
        pl.col("return_close_5").shift(-5).alias("future_return")
    ).drop_nulls()

    # Create target (1 for positive returns, -1 for negative)
    historical = historical.with_columns(
        pl.when(pl.col("future_return") > 0)
        .then(1)
        .otherwise(-1)
        .alias("target")
    )

    # Select features for the model
    feature_cols = [
        "ema_close_8", "ema_close_16", "ema_close_32",
        "rsi_close_14",
        "return_close_1", "return_close_5", "return_close_8",
        "vix_close",
        "close_volatility_20",
        "rolling_hurst_exp_126"
    ]

    # Prepare training data
    X = historical[feature_cols].to_numpy()
    y = historical["target"].to_numpy()

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train random forest
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        random_state=42
    )
    rf.fit(X_scaled, y)

    # Prepare latest data for prediction
    latest_features = latest_data[feature_cols].to_numpy()
    latest_scaled = scaler.transform(latest_features)

    # Get prediction and probability
    pred = rf.predict(latest_scaled)[0]
    prob = rf.predict_proba(latest_scaled)[0]

    # Only take positions if probability > 60%
    confidence_threshold = 0.60
    max_prob = max(prob)

    if max_prob > confidence_threshold:
        weight = float(pred)
    else:
        weight = 0.0

    return pl.DataFrame({
        "date": [latest_date],
        "weight": [weight]
    })

# Run backtest with expanding window
BACKTEST_START_DATE = pl.datetime(2015, 1, 1)

# Get sorted unique dates after start date
dates = (nq_data
    .filter(pl.col("date") >= BACKTEST_START_DATE)
    .get_column("date")
    .unique()
    .sort()
)

# Initialize empty list to store insights
all_insights = []

# Run model on each date's data slice
for date in dates:
    # Create historical slice up to and including current date
    historical_slice = nq_data.filter(pl.col("date") <= date)

    # Generate insights for this slice
    insights = random_forest_model(historical_slice)
    all_insights.append(insights)

# Combine all insights into single DataFrame
backtest_results = pl.concat(all_insights)